In [71]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [72]:
fb_df = pd.read_csv('../data/Premier-League-2015-2019.csv')


In [73]:
fb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      1520 non-null   object 
 1   HomeTeam                  1520 non-null   object 
 2   AwayTeam                  1520 non-null   object 
 3   outcome_by_HOME           1520 non-null   object 
 4   AVERAGE_ODD_WIN           1520 non-null   float64
 5   AVERAGE_ODD_DRAW          1520 non-null   float64
 6   AVERAGE_ODD_OPPONENT_WIN  1520 non-null   float64
 7   RANKINGHOME               1520 non-null   int64  
 8   RANKING AWAY              1520 non-null   int64  
 9   LAST_GAME_RHOMETEAM       1520 non-null   int64  
 10  LAST_GAME_RAWAYTEAM       1520 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 130.8+ KB


### Check for duplicates

In [74]:
fb_df.duplicated().sum()

0

### Checking for missing values

In [75]:
fb_df.isnull().sum()

Date                        0
HomeTeam                    0
AwayTeam                    0
outcome_by_HOME             0
AVERAGE_ODD_WIN             0
AVERAGE_ODD_DRAW            0
AVERAGE_ODD_OPPONENT_WIN    0
RANKINGHOME                 0
RANKING AWAY                0
LAST_GAME_RHOMETEAM         0
LAST_GAME_RAWAYTEAM         0
dtype: int64

### normalize and standardize column names

In [76]:
fb_df.rename(columns={
    'Date': 'date',
    'HomeTeam': 'home_team',
    'AwayTeam': 'away_team',
    'outcome_by_HOME': 'home_outcome',
    'AVERAGE_ODD_WIN': 'avg_odd_home_win',
    'AVERAGE_ODD_DRAW': 'avg_odd_draw',
    'AVERAGE_ODD_OPPONENT_WIN': 'avg_odd_away_win',
    'RANKINGHOME': 'home_ranking',
    'RANKING AWAY': 'away_ranking',
    'LAST_GAME_RHOMETEAM': 'last_home_result',
    'LAST_GAME_RAWAYTEAM': 'last_away_result'
}, inplace=True)

In [77]:
fb_df.head()

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
0,08/08/2015,Bournemouth,Aston Villa,L,7.024,3.303,3.748,20,17,0,0
1,08/08/2015,Chelsea,Swansea,D,6.697,4.338,8.967,1,8,0,0
2,08/08/2015,Everton,Watford,D,6.842,3.514,4.852,11,20,0,0
3,08/08/2015,Leicester,Sunderland,W,7.000,3.207,3.742,14,16,0,0
4,08/08/2015,Man United,Tottenham,W,6.477,3.651,5.318,4,5,0,0


## Label encoding

#### label encode home and away team and make sure a team in Home will have the same ID in Away

In [78]:
all_teams = pd.concat([fb_df['home_team'], fb_df['away_team']]).unique()

team_encoder = LabelEncoder()
team_encoder.fit(all_teams)

fb_df['home_team'] = team_encoder.transform(fb_df['home_team'])
fb_df['away_team'] = team_encoder.transform(fb_df['away_team'])

#### Label encoding for home_outcome

In [79]:
#label encoding for home_outcome
df_label = fb_df
df_label['home_outcome'] = df_label['home_outcome'].map({'L': 0, 'D': 1, 'W': 2})

In [80]:
df_label.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
200,12/01/2016,1,7,2,8.136,3.171,2.570,17,10,0,0
1048,04/03/2018,14,6,2,6.010,3.870,5.302,3,1,1,-1
1340,01/01/2019,0,9,2,6.606,8.222,7.491,6,20,-1,1
1390,06/02/2019,8,14,0,11.260,9.452,2.249,8,1,-1,1
121,21/11/2015,8,1,2,6.421,3.755,5.800,11,17,-1,-1
355,02/05/2016,6,23,1,7.554,3.348,2.648,1,5,-1,-1
300,20/03/2016,17,21,1,7.322,3.207,3.477,15,16,1,-1
216,16/01/2016,23,21,2,6.626,4.540,8.513,5,16,-1,-1
670,01/04/2017,24,21,2,7.017,3.392,4.305,13,17,0,0
1012,03/02/2018,3,26,2,6.436,2.967,3.420,20,11,0,0


#### One-Hot encode for home_outcome

might be useful if using differnt ml model?

In [81]:
# One-hot encoding for home_outcome
##df_onehot = fb_df
##df_onehot = pd.get_dummies(fb_df, columns=['home_outcome'])

In [82]:
df_label.sample(5)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
1383,02/02/2019,6,10,2,7.316,9.416,13.628,5,16,-1,-1
317,09/04/2016,22,6,2,8.489,3.442,2.236,8,1,1,-1
207,13/01/2016,20,18,2,7.350,3.177,3.829,9,20,0,1
1435,09/03/2019,19,23,2,7.321,7.144,2.136,17,3,-1,0
17,16/08/2015,7,0,0,9.465,3.953,2.110,10,3,0,-1


In [92]:
# have to run again to get the correct values for some weird reason
print(df_label['away_ranking'].value_counts().sort_index())

away_ranking
1      57
2      63
3      58
4      56
5      68
6      73
7      63
8      67
9      63
10     58
11     67
12     65
13     61
14     66
15     62
16     58
17     61
20    180
Name: count, dtype: int64


In [93]:
# have to run again to get the correct values for some weird reason
print(df_label['home_ranking'].value_counts().sort_index())


home_ranking
1      50
2      48
3      45
4      39
5      55
6      61
7      69
8      64
9      71
10     62
11     74
12     70
13     69
14     70
15     66
16     69
17     63
20    201
Name: count, dtype: int64


#### home and away rankings seem to be missing values for 18-19 and 20 seems to be bloated

since it seems like the rankings 18-19 might have ended up in 20. i will take all the 20s and randomly assign them 18,19 or 20

In [94]:
def redistribute_twenty(column):
    # Find where the column equals 20
    mask = df_label[column] == 20
    n = mask.sum()
    
    # Generate replacements: evenly split into 18, 19, 20
    replacements = np.array([18, 19, 20] * (n // 3 + 1))[:n]
    np.random.shuffle(replacements)

    # Apply replacements
    df_label.loc[mask, column] = replacements

# Apply to both columns
redistribute_twenty('home_ranking')
redistribute_twenty('away_ranking')


In [95]:
# have to run again to get the correct values for some weird reason
print(df_label['away_ranking'].value_counts().sort_index())

away_ranking
1     57
2     63
3     58
4     56
5     68
6     73
7     63
8     67
9     63
10    58
11    67
12    65
13    61
14    66
15    62
16    58
17    61
18    60
19    60
20    60
Name: count, dtype: int64


### Checking for outliers

In [86]:
df_label.describe()

,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
count,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000
mean,13.687500,13.687500,1.155921,7.576486,4.878582,4.741020,10.650000,10.650000,-0.067763,0.067763
std,8.070483,8.070483,0.859462,1.715519,2.011393,3.807761,5.987578,5.987578,0.852530,0.857918
min,0.000000,0.000000,0.000000,5.324000,2.935000,2.005000,1.000000,1.000000,-1.000000,-1.000000
25%,7.000000,7.000000,0.000000,6.507000,3.290750,2.491500,5.750000,5.750000,-1.000000,-1.000000
50%,14.000000,14.000000,1.000000,7.117500,3.921500,3.285500,10.500000,10.500000,0.000000,0.000000
75%,21.000000,21.000000,2.000000,8.053250,6.617250,5.095000,15.250000,15.250000,1.000000,1.000000
max,27.000000,27.000000,2.000000,18.767000,14.597000,33.015000,20.000000,20.000000,1.000000,1.000000


In [87]:
df_label.shape

(1520, 11)

#### removing outliers for "avg_odd_home_win", "avg_odd_draw" and "avg_odd_away_win"

In [88]:
def remove_outliers_iqr(df, column):
    initial_count = len(df)
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    removed_count = initial_count - len(df_filtered)
    print(f"{removed_count} rows removed from '{column}' due to outliers.")
    
    return df_filtered

In [89]:
for col in ['avg_odd_home_win', 'avg_odd_draw', 'avg_odd_away_win']:
    df_label = remove_outliers_iqr(df_label, col)

101 rows removed from 'avg_odd_home_win' due to outliers.
6 rows removed from 'avg_odd_draw' due to outliers.
167 rows removed from 'avg_odd_away_win' due to outliers.


In [90]:
X = df_label [['avg_odd_home_win', 'avg_odd_draw', 'avg_odd_away_win']]
X.describe()

,avg_odd_home_win,avg_odd_draw,avg_odd_away_win
count,1246.000000,1246.000000,1246.000000
mean,7.274838,4.441469,3.684693
std,1.043937,1.647506,1.604586
min,5.463000,2.935000,2.005000
25%,6.492000,3.250000,2.544750
50%,7.056000,3.535000,3.197000
75%,7.885250,6.383250,4.292000
max,10.355000,9.088000,9.057000


In [96]:
df_label.to_csv('../data/cleaned-premier-league-data.csv', index=False)
